# Background

<span style="color:red">You MUST run the benchmarks/scaling.ipynb for this notebook to succeed!</span>

It depends on ensembles already in storage left by that scaling benchmark.

In [ ]:
storage = '../benchmarks/scaling.h5'

# Setup

In [ ]:
import torch

print(f'{torch.version.git_version=}')
print(f'{torch.version.hip=}')
print(f'{torch.version.debug=}')
print(f'{torch.version.cuda=}')

cpu = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU {torch.cuda.current_device()}')
    torch.set_default_tensor_type('torch.cuda.DoubleTensor')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
from itertools import product

import numpy as np
import pandas as pd

import matplotlib as mpl
def dx_colors(data, color_map='rainbow'):
    spectrum = mpl.colormaps[color_map]
    
    nxed = data.groupby('nx')

    return {nx: spectrum(1-i/len(nxed)) for i, (nx, _) in enumerate(nxed)}


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
import tdg

import tdg.HMC as HMC
import tdg.plot as visualize

import h5py as h5

# Comparison

Let us try to pick a system with some physical relevance by compare against some data in [2212.05177](https://arxiv.org/abs/2212.05177).

In [ ]:
# See eq. (7.6)

def alpha3(alpha):
    
    log4 = torch.log(torch.tensor(4.))
    return 0.25 * alpha**2 * (
                    1
                    + (1.5 - log4) * alpha
                    + 3*(0.16079 - (log4-1))*alpha**2
                    # + O(alpha^3)
    )

MCDataPositive = torch.tensor([[0.561347, 0.070021],[0.473085, 0.051458],[0.382930, 0.034817],
                               [0.291280, 0.020615],[0.195786, 0.009559],[0.098755, 0.002458],])

MCDataNegative = torch.tensor([[-0.622725, 0.067143],[-0.490006, 0.047254],[-0.395163, 0.032882],
                               [-0.309663, 0.019606],[-0.255783, 0.014222],[-0.234334, 0.012622],])

In [ ]:
def comparison_plot(ax):
    x = torch.linspace(-0.65,+0.65,1000)
    ax.plot(x.cpu(), alpha3(x).cpu(), color='black')
    # ±5% for the cutoff dependence.
    ax.fill_between(x.cpu(), alpha3(0.95*x).cpu(), alpha3(1.05*x).cpu(), color='gray', alpha=0.2)
    ax.plot(MCDataNegative[:,0].cpu(), MCDataNegative[:,1].cpu(), color='gray', marker='o', linestyle='none')
    ax.plot(MCDataPositive[:,0].cpu(), MCDataPositive[:,1].cpu(), color='blue', marker='v', linestyle='none')
    ax.set_xlim([-0.65,+0.65])
    ax.set_ylim([0,0.075])    

fig, ax = plt.subplots(1,1, figsize=(10,6))
comparison_plot(ax)

# Parameter Scan

In benchmarks/scaling.ipynb we scaled towards the temporal and spatial continuum limits, holding the physics (M, a, L, β, µ, h) fixed, which fixes the ERE too.

In [ ]:
NT = (8, 16, 24, 32, 48, 64,)
NX = (7, 9, 11, 13, 15, 17, 19, 21)

mu = 0.
beta = 3. / 49
# Read the ERE in below

def key(nx, nt, root='/'):
    return f'{root}/{nx=}/{nt=}'

# Single-Ensemble Measurements

[Because of the memory costs](https://github.com/evanberkowitz/two-dimensional-gasses/issues/49) we only measure on a subset of all the ensembles and do not bin but simply keep only every n=9th configuration.

No study of errors; they may be accurate or underestimated!

In [ ]:
data = pd.DataFrame()

# Keep the memory footprint down.
NX = NX[:4]
NT = NT[:5]

for (nx, nt) in product(NX, NT):   
    try:
        with h5.File(storage, 'r') as f:
            ensemble = tdg.ensemble.GrandCanonical.from_h5(f[f'{key(nx,nt)}'])
            ere = ensemble.Action.Tuning.ere
    except:
        continue

    # Keep the number of measurements to a minimum, because of the memory costs.
    # bootstrapped = ensemble.cut(100).every(9).bootstrapped()
    bootstrapped = ensemble.bootstrapped()
    
    row = pd.Series({
        'nx': nx, 'nt': nt, 
        'N': bootstrapped.N.cpu().detach().real.numpy(),
        'C': bootstrapped.Contact.cpu().detach().real.numpy(),
        'K': bootstrapped.Kinetic.cpu().detach().real.numpy(),
        'V': bootstrapped.Potential.cpu().detach().real.numpy(),
        'F': bootstrapped.FreeEnergy.cpu().detach().real.numpy(),
    })
    data = pd.concat((data, pd.DataFrame((row,))), ignore_index=True, axis=0)
    
print('done!')

Now we add expectation-value dependent quantities under the bootstrap.

In [ ]:
data['c/kF4'] = data['C'] / (2*np.pi * data['N'])**2
data['k/kF4'] = data['K'] / (2*np.pi * data['N'])**2
data['v/kF4'] = data['V'] / (2*np.pi * data['N'])**2
data['f/kF4'] = data['F'] / (2*np.pi * data['N'])**2
data['kF a']  = data['N'].apply(lambda x: np.sqrt(x/(2*np.pi))) * ere.a.clone().cpu().detach().numpy()
data['alpha'] = - 1 / data['kF a'].apply(lambda x: np.log(x))

In [ ]:
# Styling

marker = {
    # Key is nt
    8: '^',
    16: 's',
    24: 'd',
    32: 'D',
    48: 'h',
    64: 'o',
}


color = dx_colors(data)

nxed = data.groupby('nx')
_e = 1.e-7
epsilon = {nx: _e*v for ((nx, _), v) in zip(nxed, (torch.arange(len(nxed))-(len(nxed)-1)/2).cpu())}

# Extrapolate and Compare!

We won't do any fitting; we'll simply do "chi-by-eye" to see if we are reproducing the contact density in [2212.05177](https://arxiv.org/abs/2212.05177).

In [ ]:
quantities = (
            'alpha',
            'c/kF4',
            'k/kF4',
            'v/kF4',
            'f/kF4',
            'N',
        )

fig, ax = plt.subplots(1+len(quantities),1, figsize=(10, 3*(len(quantities)+1)))

for a in ax[:-1]:
    a.set_xlim([-0.05*(beta/(min(NT)))**2, 1.05*(beta/(min(NT)))**2])

comparison_plot(ax[-1])

alpha_estimate = 0 # aggregates a value for alpha to compare the continuum limit against

for (idx, b) in data.iterrows():
    
    nx = b['nx']
    nt = b['nt']

    alpha = b['alpha'].mean()
    alpha_err = b['alpha'].std()
    alpha_estimate += alpha
    
    for quantity, axis in zip(quantities, ax[:-1]):

        mean = b[quantity].mean()
        err  = b[quantity].std()


        
        axis.errorbar(((beta/nt)**2 + epsilon[nx],), (mean,), yerr=(err,),
                   marker=marker[nt], color=color[nx], label=(f'{nx=}' if nt==8 else None)
                  )
    
    ax[-1].errorbar((alpha,), (b['c/kF4'].mean(),), xerr=(alpha_err,), yerr=(b['c/kF4'].std(),),
                   marker=marker[nt], color=color[nx], alpha=0.25
                  )

for quantity, axis in zip(quantities, ax[:-1]):
    axis.set_ylabel(quantity)
    axis.set_xlabel('(β/nt)^2')

ax[0].set_title(f'β/ML^2={beta:.4f} 2πa/L={ere.a:.4f} µ=0 h=0')
ax[0].legend()

alpha_estimate /= len(data)
cbykF4_estimate =     alpha3(     alpha_estimate)
cbykF4_estimate_err = (alpha3(1.05*alpha_estimate)-cbykF4_estimate)
ax[1].errorbar((0, ), (cbykF4_estimate.cpu().detach().numpy(),), 
               yerr=(cbykF4_estimate_err.cpu().detach().numpy(),), 
               marker='*', color='black'
              )

ax[-1].set_ylabel('c/kF^4')
ax[-1].set_xlabel('α')

plt.tight_layout()
plt.savefig('continuum-limit.pdf')